# TME 5 : Localisation de pattern (motifs) dans le genome

Lev Savolskiy
21241759

Nous allons implémenter plusieurs algorithmes afin de localiser un motif d'intérêt dans un génome.

### A -  Index of fixed length words
3\. Nous allons implémenter l'algorithme "Index of fixed length words". Faire une fonction pour indexer les positions d'occurrences de tous les mots de taille k dans un texte (génome), la fonction doit renvoyer un dictionnaire ou les clés sont les mots et les valeurs les positions dans le texte où se trouve les mots.

In [1]:
def indexTable(m, sequence):
    """
    Indexer les positions d'occurrences de tous les mots de taille k dans une sequence
    entrée m : taille du mot à chercher dans le motif m <= k
    entrée sequence : chaine de caractère représentant une sequence d'ADN
    sortie indexes : dictionaire où les clés sont les mots et les valeurs les positions dans la sequence
    """
    indexes  = {}

    for i in range(0, len(sequence)-m + 1):
        if (sequence[i:i+m] not in indexes.keys()):
            indexes[sequence[i:i+m]] = []
            indexes[sequence[i:i+m]].append(i)
        else:
            indexes[sequence[i:i+m]].append(i)

    return indexes

inds = indexTable(2, "abcdabda") #{'ab': [0, 4], 'bc': [1], 'cd': [2], 'da': [3, 6], 'bd': [5]}
print (inds)

{'ab': [0, 4], 'bc': [1], 'cd': [2], 'da': [3, 6], 'bd': [5]}


4\. Faire une fonction pour chercher toutes les occurrences d'un motif dans un génome utilisant la table des indexes de la question précédente. Nous allons permettre un nombre maximum de variations. Pour cela, implémenter ou re-utiliser la fonction distance de hamming

In [16]:
def hamming(seq1, seq2):
    """
	Calcul la distance de hamming entre deux chaînes de caractères
	entrée str1: chaîne de caractères
	entrée str2: chaîne de caractères
	sortie distance: distance de hamming
	"""
    if len(seq1) != len(seq2):
	    raise Exception("str1 = " + seq1 + ", str2 = " + seq2)
    
    return sum([1 for i in range(len(seq1)) if seq1[i].lower() != seq2[i].lower()])

def chercherWithIndexTable(m, table, sequence, motif, maxVar):
    """
    chercher les positions d'un motif dans une séquence en admettant au maximum maxVar variations
    entrée m : taille du mot à chercher dans le motif m <= k, le meme utilise pour indexer les sequences
    entrée table : dictionaire où les clés sont les mots et les valeurs les positions dans la sequence
    entrée sequence : chaine de caractère représentant une sequence d'ADN
    entrée motif : chaine de caractère représentant le motif à chercher
    entrée maxVar : le maximum variations entre le motif et un mot de taille k dans la sequence
    sortie motifPos : dictionnaire où les clés sont les motifs trouvé et les valeurs leurs positions dans la sequence.
    """
    k = len(motif)
    motifPos = {}
    if motif[0:m] in table.keys():
        for i in table[motif[0:m]]:
            if i+k < len(sequence):
                if hamming(sequence[i:i+k], motif) < maxVar:
                        if sequence[i:i+k] in motifPos.keys():
                            motifPos[sequence[i:i+k]].append(i)
                        else:
                            motifPos[sequence[i:i+k]] = []
                            motifPos[sequence[i:i+k]].append(i)


    return motifPos


m = 3; motif = "TAGTT"; maxVar = 2
inds = indexTable(m, "ACGAAATAGTTAGAA")
print(inds)
#{'acg': [0], 'cga': [1], 'gaa': [2, 12], 'aaa': [3], 'aat': [4], 'ata': [5], 'tag': [6, 10], 'agt': [7], 'gtt': [8], 'tta': [9], 'aga': [11]}
motifPos = chercherWithIndexTable(m, inds, "ACGAAATAGTTAGAA", "TAGTT", maxVar)
print(motifPos)
#{'tagtt': [6]}

{'ACG': [0], 'CGA': [1], 'GAA': [2, 12], 'AAA': [3], 'AAT': [4], 'ATA': [5], 'TAG': [6, 10], 'AGT': [7], 'GTT': [8], 'TTA': [9], 'AGA': [11]}
{'TAGTT': [6]}


5\. Tester l'algorithme **"Index of fixed length words"** sur vos données de chipSeq. Pour cela développer une fonction pour chercher un motif d'intérêt sur chaque séquence du fichier Peaks.
Voici les motifs que vous devez trouver selon le fichier de Peaks:

 - Peaks 1 Atf1 (TGCACCC / GGGTGCA)
 - Peaks 2 Atm1 (CAGCAAaaa / tttTTGCTG)
 - Peaks 3 Hap4 (CCAAT / ATTGG)
 - Peaks 4 Mac1 (GAGCAAA / TTTGCTC)
 - Peaks 5 Mac1 (GAGCAAA / TTTGCTC)
 - Peaks 6 Pdr1 (CCACGGA / TCCGTGG)
 - Peaks 7 Pdr1 (CCACGGA / TCCGTGG)

In [17]:
import time

k = 5
genome = "C_glabrata_1000bp_only.fasta"
peaks = "Sequence_by_Peaks_3.fasta" ### modifier avec votre fichier !!!
motif = "CCATT" # Le motif à localiser pour votre fichier, par example "tgcaccc"

# Fonction modifié
def readFasta(fastaFileName):
    """
    Read a fasta file
    entrée fastaFileName: nom du fichier fasta
    sortie sequences: liste contenant toutes les sequences du fichier
    """

    sequence = ""
    sequences_list = []
    prev_header = ""
    header = ""

    for line in open(fastaFileName):
        string = line.strip()
        if string[0] != ">":
            if prev_header != header:
                # print(header) # dans le cas ou on veut garde cet info
                prev_header = header
            sequence = sequence + string
        else:
            header = string
            if sequence != "":
                sequences_list.append(sequence)
                sequence = ""

    sequences_list.append(sequence)
    return sequences_list

sequencesPeaks = readFasta(peaks)
sequences1000 = readFasta(genome)


def findMotifData(sequences, motif, m, maxVar):
    """
    chercher les positions d'un motif dans un ensemble de séquence d'ADN en admettant un maximum de variations
    entrée m : taille du mot à chercher dans le motif m <= k, le meme utilise pour indexer les sequences
    entrée sequences : list contenant les sequence d'ADN
    entrée motif : chaine de caractère représentant le motif à chercher
    entrée maxVar : le maximum variations entre le motif et un mot de taille k dans la sequence
    sortie posList : list contenant les positions dans les sequences ou se trouve le motif.
    """
    count = 0; posList = []

    for i in sequences:
        inds = indexTable(m, i.upper())
        posList += chercherWithIndexTable(m, inds, i.upper(), motif, maxVar).items()


    return posList



In [18]:
posList = findMotifData(sequencesPeaks, motif, 4, 1)

print(len(posList))


42


6\. Récupérer les motifs et générer un LOGO à l'aide de l'outil WEB-LOGO https://weblogo.berkeley.edu/logo.cgi

In [19]:
print(posList)

[('CCATT', [381]), ('CCATT', [81]), ('CCATT', [586]), ('CCATT', [270, 410]), ('CCATT', [288]), ('CCATT', [16, 63]), ('CCATT', [26]), ('CCATT', [24]), ('CCATT', [650, 664]), ('CCATT', [81]), ('CCATT', [76]), ('CCATT', [170, 372, 437]), ('CCATT', [182, 339, 375, 448]), ('CCATT', [73]), ('CCATT', [175]), ('CCATT', [19]), ('CCATT', [130]), ('CCATT', [327, 499]), ('CCATT', [503]), ('CCATT', [406, 458]), ('CCATT', [270]), ('CCATT', [596]), ('CCATT', [240]), ('CCATT', [192]), ('CCATT', [66, 175]), ('CCATT', [472, 749]), ('CCATT', [366, 396, 685]), ('CCATT', [177]), ('CCATT', [66]), ('CCATT', [359]), ('CCATT', [126, 174]), ('CCATT', [213]), ('CCATT', [440, 542]), ('CCATT', [38]), ('CCATT', [200]), ('CCATT', [1]), ('CCATT', [262]), ('CCATT', [52, 87]), ('CCATT', [45]), ('CCATT', [249]), ('CCATT', [113, 252]), ('CCATT', [19])]


### B -  Matrice de fréquences
7\.Nous allons implémenter la recherche de motif par matrice de fréquence. Nous allons utiliser les matrices déjà fabriquées que vous pouvez télécharger sur le site http://jaspar.genereg.net/.
Une fois que vous avez chargé votre matrice de fréquence, vous devez la transformer en matrice de probabilité en la normalisant, diviser chaque cellule par la somme de sa colonne.

In [37]:
import numpy as np

nuc = ['A', 'C', 'G', 'T']
q = 4

#Matrice du motif Atf1, attention la matrice peut avoir plus de colonnes que le motif décrit précédemment,
#ajuster le k ou effacer les colonnes qui ne vous intéressent pas.

matFreq = [[136,716,145,34,160,10,961,8,25,19],
[78,25,31,47,10,966,9,972,955,740],
[142,31,288,39,812,11,15,9,2,13],
[641,226,535,878,16,10,14,10,16,225]]

print(matFreq)

def computing_pwm(M, cols):
    """
    Calcul la matrice de poids position à partir de la matrice de frequence
    entrée M : matrice de frequence
    sortie PWM : matrice de probabilites ou poids position
    """
    PWM = []

    for i in range(4):
        PWM.append([])
        for j in range(cols):
            PWM[i].append(0)

    for i in range(cols):
        summ = 0
        for j in range(4):
            summ += M[j][i]
        for j in range(4):
            PWM[j][i] = M[j][i] / summ

    return PWM

k = 5
PWM = computing_pwm(matFreq, k); 
print (PWM)


[[136, 716, 145, 34, 160, 10, 961, 8, 25, 19], [78, 25, 31, 47, 10, 966, 9, 972, 955, 740], [142, 31, 288, 39, 812, 11, 15, 9, 2, 13], [641, 226, 535, 878, 16, 10, 14, 10, 16, 225]]
[[0.13640922768304914, 0.717434869739479, 0.14514514514514515, 0.03406813627254509, 0.16032064128256512], [0.07823470411233702, 0.025050100200400802, 0.031031031031031032, 0.047094188376753505, 0.01002004008016032], [0.1424272818455366, 0.031062124248496994, 0.2882882882882883, 0.03907815631262525, 0.8136272545090181], [0.6429287863590772, 0.22645290581162325, 0.5355355355355356, 0.8797595190380761, 0.01603206412825651]]


8\. Déterminer les paramètres f(0)(b) du modèle nul, où
\begin{equation}
f^{(0)}(b) = \frac 1L \sum_{i=0}^{L-1} \omega_i(b)\ ,
\end{equation}

In [82]:
def f0_calcule(PWM, L):
    """
    Calcul les valeurs de probabilites d'un modele independant de positions (modele Null)
    entrée PWM : matrice de probabilites ou poids positions
    sortie  f_0 : vecteur contenant un modele independant de positions (modele Null)
    """
    f_0 = []
    for i in range(4):
        f0temp = 0
        for j in range(L):
            f0temp += PWM[i][j]
        f0temp /= L
        f_0.append(f0temp)

    return f_0

f_0 = f0_calcule(PWM, k)
print (f_0)

[0.23867560402455665, 0.03828601276013653, 0.26289662104079303, 0.46014176217451375]


9\. Faites une fonction pour calculer log-rapport de vraisemblancee d'une sequence de taille k et la matrice poids position.
\begin{equation}
\label{eq:ll}
\ell(b_0,...,b_{k-1}) = \log_2 \frac {P(b_0,...,b_{k-1} | \omega )
}{P^{(0)}(b_0,...,b_{k-1})}
= \sum_{i=0}^{k-1} \log_2 \frac {\omega_i(b_i)}{f^{(0)}(b_i)}\ .
\end{equation}

In [83]:
import math

def loglikehood(seq, PWM, f_0, L):
	"""
	Calcul le rapport de vraissemblance entre une sequence et une matrice de poids position
	entrée PWM : matrice de probabilites ou poids positions
	entrée f_0 : vecteur contenant le modele independant de positions (modele Null)
	entrée seq : une sequence d'ADN de taille k, ou k est le nombre de colonnes de PWM
	sortie ll : rapport de vraissemblance
	"""
	L_0 = len(seq)
	nuc = {'A' : 0, 'C' : 1, 'G' : 2, 'T' : 3}
	l_win = 0

	for i in range(L): 
		l_win += math.log2(PWM[nuc[seq[i]]][i]/f_0[nuc[seq[i]]])
	
	return l_win

loglikehood('acctgcaccc'.upper(), PWM, f_0, k)

0.8426865225103377

10\. Faire une fonction que pour chaque séquence cherche la/les positions avec une log-vraisemblance positive.

In [90]:
def searchPWM(sequences, k, PWM, f_0):
    """
    Cherche les positions dans un ensemble de séquence qui maxime le rapport de vraisemblance
    entrée sequences : ensemble de séquence d'ADN
    entrée k : nombre de colonnes d'PWM
    entrée PWM : matrice de probabilités ou poids positions
    entrée f_0 : vecteur contenant le modèle indépendant de positions (modèle Null)
    sortie posList: liste contenant pour chaque séquence la/les positions ayant un rapport de vraisemblance positive
    """
    posList = []; foundMotifPWM = []

    for i in range(len(sequences)):
        maxx = 0
        foundMotifPWM.append("")
        posList.append("")
        for j in range(0, len(sequences[i])-k):
            if loglikehood(sequences[i][j:j+k].upper(), PWM, f_0, k) > maxx:
                posList[i] = j
                foundMotifPWM[i] = sequences[i][j:j+k]
                maxx = loglikehood(sequences[i][j:j+k].upper(), PWM, f_0, k)

    return posList, foundMotifPWM


posList, motifs = searchPWM(sequencesPeaks, k, PWM, f_0)
print(posList)


[382, 82, 156, 368, 183, 271, 231, 7, 64, 49, 68, 76, 56, 354, 665, 16, 401, 19, 106, 122, 171, 129, 493, 222, 118, 110, 253, 164, 38, 44, 95, 190, 405, 151, 278, 21, 9, 66, 115, 16, 4, 564, 439, 158, 190, 119, 277, 53, 454, 179, 246, 329, 27, 67, 51, 276, 241, 26, 219, 94, 367, 448, 611, 183, 169, 143, 322, 237, 257, 127, 344, 1, 88, 54, 92, 67, 88, 138, 63, 125, 62, 94, 98, 2, 61, 83, 32, 146, 288, 263, 80, 53, 175, 39, 163, 125, 66, 18, 136, 224, 61, 39, 112, 86, 2, 10, 227, 215, 100]


11\.  (Bonus) Certains motifs si chevauche dans ce cas il faut choisir ceux qui ont la meilleur  log-vraisemblance. Par exemple pour une taille de motif k = 7, si vous avez deux motifs, un qui commence dans la position 45 et un autre en position 47, garder celui avec la log-vraisemblance la plus grande.

In [ ]:
def searchPWMOptmiseMotifs(sequences, k, PWM, f_0):
    """
    Cherche les positions dans un ensemble de séquence qui maxime le rapport de vraisemblance et elimine les motifs chevauchante
    entrée sequences : ensemble de séquence d'ADN
    entrée k : nombre de colonnes d'PWM
    entrée PWM : matrice de probabilités ou poids positions
    entrée f_0 : vecteur contenant le modèle indépendant de positions (modèle Null)
    sortie posList: liste contenant pour chaque séquence la/les positions ayant un rapport de vraisemblance positive

    """
    
    
    return None

12\.Tester l'algorithme sur vos données de chipSeq et dessiner le LOGO